# Classification : Prediction du churn client

> Ce notebook est un exemple pratique clef en main pour apprendre les techniques de classification en Machine Learning.
> Nous allons predire si un client va quitter l'entreprise (churn) a partir de ses caracteristiques comportementales.
>
> **Objectifs :**
> - Maitriser le pipeline complet d'un projet de classification binaire
> - Comparer plusieurs algorithmes (Regression Logistique, Random Forest, XGBoost)
> - Comprendre les metriques de classification (accuracy, precision, recall, F1, AUC)
> - Optimiser un modele avec GridSearchCV et Pipeline sklearn

In [ ]:
# ============================================================
# Cellule 1 : Imports et chargement des donnees
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    roc_auc_score,
    precision_recall_curve,
    ConfusionMatrixDisplay,
)

import warnings
warnings.filterwarnings("ignore")

# Configuration graphique
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["figure.dpi"] = 100
sns.set_style("whitegrid")

# Chargement des donnees
df = pd.read_csv("../data/clients_churn.csv")

print(f"Dataset charge avec succes : {df.shape[0]} lignes, {df.shape[1]} colonnes")
df.head(10)

In [ ]:
# ============================================================
# Cellule 2 : Analyse exploratoire (EDA)
# ============================================================

print("=" * 60)
print("INFORMATIONS GENERALES")
print("=" * 60)
print(f"\nDimensions : {df.shape}")
print(f"\nTypes des colonnes :")
print(df.dtypes)

print("\n" + "=" * 60)
print("VALEURS MANQUANTES")
print("=" * 60)
print(df.isnull().sum())

print("\n" + "=" * 60)
print("STATISTIQUES DESCRIPTIVES")
print("=" * 60)
print(df.describe().round(2))

# Equilibre des classes
print("\n" + "=" * 60)
print("EQUILIBRE DES CLASSES (variable cible : churn)")
print("=" * 60)
churn_counts = df["churn"].value_counts()
churn_pct = df["churn"].value_counts(normalize=True) * 100

for val in churn_counts.index:
    label = "Churn" if val == 1 else "Non-churn"
    print(f"  {label} ({val}) : {churn_counts[val]} clients ({churn_pct[val]:.1f}%)")

## Visualisations exploratoires

Nous allons explorer les distributions des variables selon le statut churn/non-churn
pour identifier les facteurs discriminants.

In [ ]:
# ============================================================
# Cellule 3 : Visualisations
# ============================================================

# --- Taux de churn par variable categorielle ---
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for i, col in enumerate(["canal_acquisition", "type_contrat", "satisfaction_score"]):
    taux_churn = df.groupby(col)["churn"].mean().sort_values(ascending=False)
    taux_churn.plot(kind="bar", ax=axes[i], color="#e74c3c", edgecolor="black", alpha=0.8)
    axes[i].set_title(f"Taux de churn par {col}")
    axes[i].set_ylabel("Taux de churn")
    axes[i].set_ylim(0, 1)
    axes[i].tick_params(axis="x", rotation=45)
    # Ajout des valeurs sur les barres
    for j, v in enumerate(taux_churn):
        axes[i].text(j, v + 0.02, f"{v:.0%}", ha="center", fontweight="bold")

plt.tight_layout()
plt.show()

# --- Boxplots des variables numeriques par churn ---
variables_num = ["age", "revenu_mensuel", "anciennete_mois", "nb_achats", "montant_total", "nb_reclamations"]

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, col in enumerate(variables_num):
    sns.boxplot(
        data=df, x="churn", y=col, ax=axes[i],
        palette={0: "#2ecc71", 1: "#e74c3c"}
    )
    axes[i].set_title(f"{col} par statut churn")
    axes[i].set_xticklabels(["Non-churn", "Churn"])

plt.tight_layout()
plt.show()

## Preprocessing des donnees

Etapes de preparation :
1. **Encodage** des variables categorielles (`canal_acquisition`, `type_contrat`)
2. **Standardisation** des variables numeriques (necessaire pour la regression logistique)
3. **Split** train/test (80/20 avec stratification)

In [ ]:
# ============================================================
# Cellule 4 : Preprocessing (encodage, scaling, split)
# ============================================================

# Copie du dataframe
df_model = df.copy()

# Encodage des variables categorielles (One-Hot)
df_model = pd.get_dummies(df_model, columns=["canal_acquisition", "type_contrat"], drop_first=True)

print("Colonnes apres encodage :")
print(df_model.columns.tolist())

# Separation features / target
X = df_model.drop("churn", axis=1)
y = df_model["churn"]

# Split train/test avec stratification (pour conserver les proportions de churn)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Standardisation des variables numeriques
scaler = StandardScaler()
colonnes_num = ["age", "revenu_mensuel", "anciennete_mois", "nb_achats", "montant_total", "nb_reclamations", "satisfaction_score"]

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[colonnes_num] = scaler.fit_transform(X_train[colonnes_num])
X_test_scaled[colonnes_num] = scaler.transform(X_test[colonnes_num])

print(f"\nTaille du jeu d'entrainement : {X_train.shape[0]} echantillons")
print(f"Taille du jeu de test : {X_test.shape[0]} echantillons")
print(f"Nombre de features : {X_train.shape[1]}")
print(f"\nDistribution churn dans le train : {y_train.value_counts(normalize=True).to_dict()}")
print(f"Distribution churn dans le test  : {y_test.value_counts(normalize=True).to_dict()}")

## Modele 1 : Regression logistique

La regression logistique est le modele de reference pour la classification binaire.
Simple et interpretable, c'est toujours un bon point de depart.

## Comprendre les metriques de classification

Avant d'evaluer nos modeles, comprenons ce que chaque metrique signifie. L'analogie la plus parlante est celle du **test medical** :

### L'analogie du test medical

Imaginez un test de depistage pour une maladie. Le test peut donner 4 resultats :

```
                    Le test dit "Malade"    Le test dit "Sain"
Vraiment malade  →  ✅ Vrai Positif (TP)    ❌ Faux Negatif (FN)
                    "Bien detecte !"        "Rate ! Danger !"

Vraiment sain    →  ❌ Faux Positif (FP)    ✅ Vrai Negatif (TN)
                    "Fausse alarme"         "Bien identifie"
```

Dans notre cas de **churn client** :
- **Positif** = le client va partir (churn)
- **Negatif** = le client reste fidele

### Les metriques expliquees avec des mots simples

| Metrique | Question qu'elle pose | Analogie medicale | Notre cas churn |
|----------|----------------------|-------------------|-----------------|
| **Accuracy** | "Quel % de predictions sont correctes ?" | "Quel % de diagnostics sont bons ?" | 85% → 85 clients sur 100 bien classes |
| **Precision** | "Quand je dis 'positif', ai-je raison ?" | "Quand le test dit 'malade', est-ce vrai ?" | 70% → sur 10 clients predits churn, 7 partent vraiment |
| **Recall** | "Est-ce que je detecte tous les positifs ?" | "Est-ce qu'on detecte tous les malades ?" | 80% → sur 10 clients qui partent, on en detecte 8 |
| **F1-Score** | "Compromis entre precision et recall" | "Le test est-il fiable ET exhaustif ?" | Moyenne harmonique des deux |
| **AUC-ROC** | "Le modele sait-il distinguer les classes ?" | "Le test distingue-t-il bien malades et sains ?" | 0.5 = hasard, 1.0 = parfait |

### Pourquoi l'accuracy ne suffit pas ?

Imaginez : 95% de vos clients sont fideles, seulement 5% partent.
Un modele qui predit **toujours "fidele"** a **95% d'accuracy** ! Mais il ne detecte **aucun** churn. C'est un modele inutile.

> **Regle d'or** : Quand les classes sont desequilibrees (beaucoup plus de "non" que de "oui"), l'accuracy est trompeuse. Privilegiez le **F1-Score** et l'**AUC-ROC**.

### Matrice de confusion : la carte d'identite du modele

C'est le tableau qui resume TOUTES les predictions du modele :

```
                    Predit "reste"    Predit "churn"
Reste vraiment   →      TN               FP (fausse alarme)
Part vraiment    →      FN (rate !)      TP (bien detecte)
```

**Ce qu'on veut** : beaucoup de TP et TN (la diagonale), peu de FP et FN.

In [ ]:
# ============================================================
# Cellule 5 : Regression logistique + matrice de confusion
# ============================================================

def evaluer_classification(modele, X_test, y_test, nom_modele, afficher_matrice=True):
    """Fonction utilitaire pour evaluer un modele de classification."""
    y_pred = modele.predict(X_test)
    y_proba = modele.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)

    print(f"\n{'=' * 50}")
    print(f"  RESULTATS : {nom_modele}")
    print(f"{'=' * 50}")
    print(f"  Accuracy  : {acc:.4f}")
    print(f"  Precision : {prec:.4f}")
    print(f"  Recall    : {rec:.4f}")
    print(f"  F1-Score  : {f1:.4f}")
    print(f"  AUC-ROC   : {auc:.4f}")

    print(f"\n--- Rapport de classification ---")
    print(classification_report(y_test, y_pred, target_names=["Non-churn", "Churn"]))

    if afficher_matrice:
        fig, ax = plt.subplots(figsize=(6, 4))
        ConfusionMatrixDisplay.from_estimator(
            modele, X_test, y_test,
            display_labels=["Non-churn", "Churn"],
            cmap="Blues", ax=ax
        )
        ax.set_title(f"Matrice de confusion - {nom_modele}")
        plt.tight_layout()
        plt.show()

    return {
        "Modele": nom_modele,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1-Score": f1,
        "AUC-ROC": auc,
    }


# Entrainement de la regression logistique
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_scaled, y_train)

# Evaluation
resultats = []
resultats.append(evaluer_classification(lr, X_test_scaled, y_test, "Regression Logistique"))

## Modele 2 : Random Forest

Le Random Forest est un modele d'ensemble base sur des arbres de decision.
Il nous permet aussi d'obtenir l'importance des features.

In [ ]:
# ============================================================
# Cellule 6 : Random Forest + importance des features
# ============================================================

# Entrainement du Random Forest (pas besoin de scaling pour les arbres)
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1,
)
rf.fit(X_train, y_train)

# Evaluation (sur donnees non scalees car arbres)
resultats.append(evaluer_classification(rf, X_test, y_test, "Random Forest"))

# Importance des features
print("\n--- Importance des features ---")
importances = pd.DataFrame({
    "Feature": X_train.columns,
    "Importance": rf.feature_importances_
}).sort_values("Importance", ascending=False)

fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(data=importances, x="Importance", y="Feature", palette="RdYlGn_r", ax=ax)
ax.set_title("Importance des features - Random Forest")
ax.set_xlabel("Importance")
ax.set_ylabel("")
plt.tight_layout()
plt.show()

## Modele 3 : Gradient Boosting (equivalent XGBoost avec sklearn)

Le Gradient Boosting est un algorithme d'ensemble qui construit les arbres de maniere sequentielle,
chaque arbre corrigeant les erreurs du precedent. Nous utilisons ici `GradientBoostingClassifier`
de sklearn qui offre des performances similaires a XGBoost.

In [ ]:
# ============================================================
# Cellule 7 : Gradient Boosting (XGBoost-like)
# ============================================================

gb = GradientBoostingClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
)
gb.fit(X_train, y_train)

# Evaluation
resultats.append(evaluer_classification(gb, X_test, y_test, "Gradient Boosting"))

## Comparaison des courbes ROC

### C'est quoi une courbe ROC ?

Imaginez un **curseur** qu'on fait glisser : a gauche, le modele est tres prudent (peu de fausses alarmes mais rate des churns). A droite, le modele est tres sensible (detecte tout mais beaucoup de fausses alarmes).

La courbe ROC trace **toutes ces positions du curseur** :
- **Axe X** = Taux de fausses alarmes (FPR) → on veut le MINIMUM
- **Axe Y** = Taux de detection (TPR/Recall) → on veut le MAXIMUM

**Comment lire le graphique ?**
- La ligne en pointilles = un modele qui tire a pile ou face (AUC = 0.5)
- Plus la courbe est proche du **coin superieur gauche**, meilleur est le modele
- **AUC** (Area Under Curve) = l'aire sous la courbe. Plus c'est proche de 1, mieux c'est

| AUC | Interpretation |
|-----|---------------|
| 0.50 | Le modele tire au hasard |
| 0.60-0.70 | Mediocre |
| 0.70-0.80 | Acceptable |
| 0.80-0.90 | Bon |
| 0.90-1.00 | Excellent |

In [ ]:
# ============================================================
# Cellule 8 : Courbes ROC superposees
# ============================================================

fig, ax = plt.subplots(figsize=(8, 6))

# Modeles et leurs donnees de test correspondantes
modeles_roc = [
    (lr, X_test_scaled, "Regression Logistique", "#3498db"),
    (rf, X_test, "Random Forest", "#2ecc71"),
    (gb, X_test, "Gradient Boosting", "#e74c3c"),
]

for modele, X_eval, nom, couleur in modeles_roc:
    y_proba = modele.predict_proba(X_eval)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc = roc_auc_score(y_test, y_proba)
    ax.plot(fpr, tpr, label=f"{nom} (AUC = {auc:.3f})", color=couleur, linewidth=2)

# Ligne de reference (modele aleatoire)
ax.plot([0, 1], [0, 1], "k--", linewidth=1, label="Modele aleatoire (AUC = 0.500)")

ax.set_xlabel("Taux de faux positifs (FPR)")
ax.set_ylabel("Taux de vrais positifs (TPR / Recall)")
ax.set_title("Courbes ROC - Comparaison des modeles")
ax.legend(loc="lower right")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Ajustement du seuil de decision

### C'est quoi le seuil ?

Le modele ne predit pas directement "churn" ou "pas churn". Il donne une **probabilite** : "ce client a 73% de chances de partir".

Le **seuil** est la limite a partir de laquelle on decide : si la probabilite depasse le seuil, on dit "churn".

Par defaut, le seuil est a **0.5** (50%). Mais on peut le bouger :

```
Seuil = 0.3 (prudent)         Seuil = 0.7 (strict)
─────────────────              ─────────────────
"Des que c'est > 30%,          "Seulement si c'est > 70%,
 je considere churn"            je considere churn"

→ Detecte PLUS de churns       → Detecte MOINS de churns
→ Mais plus de fausses alarmes  → Mais chaque alerte est fiable
→ Meilleur Recall               → Meilleure Precision
```

### Quel seuil choisir ?

Ca depend du **cout metier** :

| Situation | Seuil recommande | Pourquoi |
|-----------|-----------------|----------|
| Detecter une maladie grave | **Bas** (0.2-0.3) | Mieux vaut une fausse alarme que rater un malade |
| Envoyer une promotion couteuse | **Haut** (0.6-0.7) | Chaque envoi coute cher, on veut cibler juste |
| Detecter le churn client | **Moyen** (0.4-0.5) | Equilibre entre cout de retention et perte de client |

> **A retenir** : Il n'y a pas de "bon" seuil universel. Le F1-Score optimal trouve le meilleur compromis, mais la decision finale depend du contexte metier.

In [ ]:
# ============================================================
# Cellule 9 : Ajustement du seuil + courbe precision-recall
# ============================================================

# Utilisation du meilleur modele pour l'analyse du seuil
meilleur_modele = gb  # Gradient Boosting
y_proba = meilleur_modele.predict_proba(X_test)[:, 1]

# Courbe precision-recall
precision_vals, recall_vals, seuils_pr = precision_recall_curve(y_test, y_proba)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Courbe Precision-Recall
axes[0].plot(recall_vals, precision_vals, color="#8e44ad", linewidth=2)
axes[0].set_xlabel("Recall")
axes[0].set_ylabel("Precision")
axes[0].set_title("Courbe Precision-Recall")
axes[0].grid(True, alpha=0.3)
axes[0].set_xlim(0, 1)
axes[0].set_ylim(0, 1.05)

# 2. Precision et Recall en fonction du seuil
axes[1].plot(seuils_pr, precision_vals[:-1], label="Precision", color="#3498db", linewidth=2)
axes[1].plot(seuils_pr, recall_vals[:-1], label="Recall", color="#e74c3c", linewidth=2)

# Calcul du F1 pour chaque seuil
f1_seuils = 2 * (precision_vals[:-1] * recall_vals[:-1]) / (precision_vals[:-1] + recall_vals[:-1] + 1e-10)
axes[1].plot(seuils_pr, f1_seuils, label="F1-Score", color="#2ecc71", linewidth=2, linestyle="--")

# Seuil optimal (max F1)
idx_optimal = np.argmax(f1_seuils)
seuil_optimal = seuils_pr[idx_optimal]
axes[1].axvline(x=seuil_optimal, color="gray", linestyle=":", linewidth=2, label=f"Seuil optimal = {seuil_optimal:.2f}")

axes[1].set_xlabel("Seuil de decision")
axes[1].set_ylabel("Score")
axes[1].set_title("Precision / Recall / F1 en fonction du seuil")
axes[1].legend(loc="center left")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Comparaison seuil 0.5 vs seuil optimal
print(f"\n--- Comparaison des seuils ---")
print(f"  Seuil par defaut : 0.50")
print(f"  Seuil optimal (max F1) : {seuil_optimal:.2f}")

for seuil in [0.5, seuil_optimal]:
    y_pred_seuil = (y_proba >= seuil).astype(int)
    prec = precision_score(y_test, y_pred_seuil)
    rec = recall_score(y_test, y_pred_seuil)
    f1 = f1_score(y_test, y_pred_seuil)
    print(f"\n  Seuil = {seuil:.2f} -> Precision={prec:.3f}, Recall={rec:.3f}, F1={f1:.3f}")

## Pipeline + GridSearchCV

En production, il est recommande d'utiliser des **Pipelines sklearn** qui enchainent
preprocessing et modele dans un seul objet. Cela evite les fuites de donnees (data leakage)
et simplifie le deploiement.

Nous allons aussi utiliser **GridSearchCV** pour optimiser les hyperparametres.

In [ ]:
# ============================================================
# Cellule 10 : Pipeline + GridSearchCV
# ============================================================

# Construction du pipeline : Scaling -> Random Forest
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("classifier", RandomForestClassifier(random_state=42, n_jobs=-1)),
])

# Grille d'hyperparametres a tester
param_grid = {
    "classifier__n_estimators": [50, 100, 200],
    "classifier__max_depth": [5, 10, None],
    "classifier__min_samples_split": [2, 5],
}

print("Lancement du GridSearchCV...")
print(f"Nombre de combinaisons : {3 * 3 * 2} x 5 folds = {3 * 3 * 2 * 5} evaluations")

# Recherche des meilleurs hyperparametres
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring="f1",
    n_jobs=-1,
    verbose=0,
)
grid_search.fit(X_train, y_train)

print(f"\n--- Resultats du GridSearchCV ---")
print(f"  Meilleurs hyperparametres : {grid_search.best_params_}")
print(f"  Meilleur score F1 (CV)    : {grid_search.best_score_:.4f}")

# Evaluation du meilleur modele sur le jeu de test
print("\n--- Evaluation sur le jeu de test ---")
y_pred_best = grid_search.predict(X_test)
y_proba_best = grid_search.predict_proba(X_test)[:, 1]

print(f"  Accuracy  : {accuracy_score(y_test, y_pred_best):.4f}")
print(f"  Precision : {precision_score(y_test, y_pred_best):.4f}")
print(f"  Recall    : {recall_score(y_test, y_pred_best):.4f}")
print(f"  F1-Score  : {f1_score(y_test, y_pred_best):.4f}")
print(f"  AUC-ROC   : {roc_auc_score(y_test, y_proba_best):.4f}")

# Affichage des resultats de la grille triee par score
print("\n--- Top 5 des combinaisons ---")
resultats_grid = pd.DataFrame(grid_search.cv_results_)
top5 = resultats_grid.nsmallest(5, "rank_test_score")[["params", "mean_test_score", "std_test_score"]]
for idx, row in top5.iterrows():
    print(f"  F1={row['mean_test_score']:.4f} (+/-{row['std_test_score']:.4f}) | {row['params']}")

## Conclusion

### Recapitulatif des resultats

Nous avons compare 3 modeles de classification pour predire le churn client :

| Modele | Avantages | Inconvenients |
|--------|-----------|---------------|
| **Regression Logistique** | Simple, interpretable, rapide | Suppose linearite |
| **Random Forest** | Non lineaire, robuste, importance features | Plus lent, moins interpretable |
| **Gradient Boosting** | Tres performant, flexible | Risque de surapprentissage, lent |

### Points cles

1. Les **reclamations** et le **score de satisfaction** sont les meilleurs predicteurs du churn
2. Les clients **Web** et **CDD/Freelance** ont un taux de churn plus eleve
3. L'**anciennete** et le **montant total** d'achats sont negativement correles au churn
4. L'ajustement du **seuil de decision** permet d'adapter le modele au contexte metier

### Pour aller plus loin

- Gerer le **desequilibre des classes** (SMOTE, class_weight)
- Tester des modeles plus avances (XGBoost, LightGBM, CatBoost)
- Implementer un **systeme d'alerte** pour les clients a risque
- Calculer la **valeur metier** : combien un modele de churn fait-il economiser ?

### Lexique debutant

| Terme | Definition simple |
|-------|------------------|
| **Classification** | Predire une categorie (oui/non, spam/pas spam, churn/fidele) |
| **Churn** | Un client qui quitte l'entreprise ou arrete d'acheter |
| **Matrice de confusion** | Tableau qui resume les 4 types de resultats (TP, TN, FP, FN) |
| **Precision** | "Quand je dis oui, ai-je raison ?" |
| **Recall** | "Est-ce que je detecte tous les oui ?" |
| **F1-Score** | Compromis entre precision et recall (utile quand les classes sont desequilibrees) |
| **AUC-ROC** | Score global de 0.5 (hasard) a 1 (parfait) |
| **Seuil** | Probabilite a partir de laquelle on decide "positif" (defaut : 0.5) |
| **Stratification** | S'assurer que train et test ont les memes proportions de chaque classe |
| **GridSearchCV** | Tester automatiquement plusieurs combinaisons d'hyperparametres |

In [ ]:
# ============================================================
# Cellule 11 : Resume final
# ============================================================

print("=" * 60)
print("  TABLEAU COMPARATIF FINAL")
print("=" * 60)

df_resultats = pd.DataFrame(resultats)
df_resultats = df_resultats.sort_values("F1-Score", ascending=False)
print(df_resultats.to_string(index=False, float_format="{:.4f}".format))

# Meilleur modele
meilleur = df_resultats.iloc[0]
print(f"\n{'=' * 60}")
print(f"  MEILLEUR MODELE : {meilleur['Modele']}")
print(f"{'=' * 60}")
print(f"  F1-Score : {meilleur['F1-Score']:.4f}")
print(f"  AUC-ROC  : {meilleur['AUC-ROC']:.4f}")
print(f"  Recall   : {meilleur['Recall']:.4f}")
print(f"\n  Interpretation :")
print(f"  Le modele detecte {meilleur['Recall']*100:.0f}% des clients qui vont churner")
print(f"  avec une precision de {meilleur['Precision']*100:.0f}%.")
print("\n" + "=" * 60)